# Интеллектуальный анализ данных – весна 2025
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.
* Если в задании есть вопрос на рассуждение, то за отсутствие ответа на него балл за задание будет снижен вполовину.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [1]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

from nltk import SnowballStemmer

In [2]:
df = pd.read_csv('tweets_coronavirus.csv', encoding='latin-1')
df.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная или очень положительная эмоциональная окраска и 0 - если отрицательная или очень отрицательная.

In [3]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'Positive' or x == 'Extremely Positive' else 0)
df.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


Сбалансированы ли классы?

In [4]:
df['Sentiment'].value_counts()

Sentiment
1    18046
0    15398
Name: count, dtype: int64

**Ответ:** более-менее сбалансированы, для наших задач вполне подойдёт

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [5]:
df[df.isna().any(axis=1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
11,3813,48765,NaN,16-03-2020,ADARA Releases COVID-19 Resource Center for Tr...,1
16,3821,48773,NaN,16-03-2020,We have AMAZING CHEAP DEALS! FOR THE #COVID201...,1
17,3822,48774,NaN,16-03-2020,We have AMAZING CHEAP DEALS! FOR THE #COVID201...,1
...,...,...,...,...,...,...
33431,44938,89890,NaN,14-04-2020,Hello everyone \r\r\nPlease share this in your...,1
33437,44947,89899,NaN,14-04-2020,UV light Sterilizer Sanitizer for your mask an...,1
33440,44950,89902,NaN,14-04-2020,@MrSilverScott you are definitely my man. I fe...,1
33441,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,0


In [6]:
df.fillna('Unknown', inplace=True)
df.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,Unknown,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,Unknown,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


Разделите данные на обучающие и тестовые в соотношении 7 : 3 и укажите `random_state=0`

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=0)

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведения токенов в нижний регистр.

In [8]:
tweets = ' '.join(train['OriginalTweet'])
display(tweets[:100])
token_list = [x.lower() for x in tweets.split()]
display(token_list[:5])

'Why we still want to buy so much stuff during quarantine https://t.co/1m881CwFUv #shopping #Covid_19'

['why', 'we', 'still', 'want', 'to']

In [9]:
from collections import Counter

token_dict = Counter(token_list)
token_dict

Counter({'the': 26815,
         'to': 23373,
         'and': 14684,
         'of': 13012,
         'a': 11737,
         'in': 11198,
         'for': 8566,
         '#coronavirus': 8223,
         'is': 7383,
         'are': 7050,
         'you': 5467,
         'on': 5452,
         'i': 5340,
         'at': 4642,
         'this': 4581,
         'with': 4063,
         'prices': 3891,
         'food': 3820,
         'we': 3787,
         'have': 3770,
         'that': 3741,
         'as': 3694,
         'be': 3570,
         'grocery': 3469,
         'supermarket': 3288,
         'people': 3175,
         'covid-19': 3173,
         'store': 3155,
         'it': 3150,
         'from': 3045,
         'all': 2808,
         'your': 2784,
         'will': 2726,
         'not': 2714,
         '#covid19': 2471,
         'our': 2460,
         'my': 2445,
         '&amp;': 2314,
         'they': 2309,
         'has': 2304,
         'consumer': 2245,
         'by': 2236,
         'or': 2234,
         '

Какой размер словаря получился?

In [10]:
len(token_dict)

79755

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе.

In [11]:
token_dict.most_common(10)

[('the', 26815),
 ('to', 23373),
 ('and', 14684),
 ('of', 13012),
 ('a', 11737),
 ('in', 11198),
 ('for', 8566),
 ('#coronavirus', 8223),
 ('is', 7383),
 ('are', 7050)]

**Ответ:** очевидно, что топ состоит из артиклей, союзов, предлогов, частиц и всякого такого, что встречается в любом предложении независимо от его смысла

Выделяется только #coronavirus, и это уже особенность нашей выборки, поскольку мы брали тексты именно о коронавирусе

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [12]:
import nltk
nltk.download("stopwords", quiet=True)

True

In [13]:
from nltk.corpus import stopwords

for word in stopwords.words('english'):
    del token_dict[word]

token_dict.most_common(10)

[('#coronavirus', 8223),
 ('prices', 3891),
 ('food', 3820),
 ('grocery', 3469),
 ('supermarket', 3288),
 ('people', 3175),
 ('covid-19', 3173),
 ('store', 3155),
 ('#covid19', 2471),
 ('&amp;', 2314)]

**Ответ:**  видим, что волновало людей на самом деле: цены, еда, магазины. В целом соотносится с впечатлениями от той эпохи

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше, выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [14]:
token_dict.most_common()[-20:]

[('skellig', 1),
 ('coast!', 1),
 ('closer!)', 1),
 ('@skelligsix18', 1),
 ('#skelligcoast2kms', 1),
 ('#southkerry', 1),
 ('https://t.co/zjcl195vqs', 1),
 ('@srinivasiyc', 1),
 ('https://t.co/iaek4fwsgz', 1),
 ('premiership', 1),
 ('non-playing', 1),
 ('subsidise', 1),
 ('playersã\x82â\x92', 1),
 ('renewing', 1),
 ('wage!', 1),
 ('flew', 1),
 ('nothing...', 1),
 ('@torontopearson', 1),
 ('@680news', 1),
 ('https://t.co/7j2y3rsld9', 1)]

**Ответ:** это вполне нормально, что некоторые слова встречаются по одному разу. Конечно, много информации они нам не дадут, но и мешать не станут, модель всё равно будет работать с более распространёнными

Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [15]:
from nltk.tokenize import TweetTokenizer

tw_token_list = TweetTokenizer(preserve_case=False).tokenize(tweets) # В задании не было сказано не учитывать капитализацию, но было бы честно не учитывать
tw_token_dict = Counter(tw_token_list)
tw_token_dict.most_common(10)

[('the', 26993),
 ('.', 24108),
 ('to', 23478),
 (',', 17571),
 ('and', 14825),
 ('of', 13044),
 ('a', 11891),
 ('in', 11348),
 ('?', 9524),
 ('#coronavirus', 8808)]

**Ответ:** TweetTokenizer выделяет пунктуацию в отдельные токены, в отличие от наивного метода, из-за чего она прибавилась к нашим частицам, артиклям и т.д.

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [16]:
from string import punctuation

stop_words = set(stopwords.words('english') + list(punctuation))
for word in stop_words:
    del tw_token_dict[word]

tw_token_dict.most_common(10)

[('#coronavirus', 8808),
 ('â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('covid', 6253),
 ('prices', 4601),
 ('\x92', 4372),
 ('food', 4367),
 ('store', 3877),
 ('supermarket', 3805)]

**Ответ:** несколько слов, конечно, есть, но вместе с ними появились какие-то символы в кодировке ASCII, которые почему-то не отображаются (а ещё едва ли вторым самым частым токеном является странная буква А)

Скорее всего в некоторых топах были неотображаемые символы или отдельные буквы не латинского алфавита. Уберем их: удалите из словаря токены из одного символа, позиция которого в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [17]:
keys_to_remove = []
for key in tw_token_dict:
    if len(key) == 1 and ord(key) >= 128:
        keys_to_remove.append(key)

for key in keys_to_remove:
        del tw_token_dict[key]

In [18]:
tw_token_dict.most_common(10)

[('#coronavirus', 8808),
 ('19', 7167),
 ('covid', 6253),
 ('prices', 4601),
 ('food', 4367),
 ('store', 3877),
 ('supermarket', 3805),
 ('grocery', 3523),
 ('people', 3463),
 ('#covid19', 2589)]

Тут всё супер, по сравнению с наивным топом отделилось число 19, которое, видимо, до этого использовалось в составе COVID19, covid-19 и прочих, и ушёл аскишный амперасант, что и к лучшему

In [19]:
tw_token_dict.most_common()[-20:]

[('https://t.co/LW1r0Rm7XS', 1),
 ('https://t.co/5cBLIqZX7L', 1),
 ('now.when', 1),
 ('milion', 1),
 ('skellig', 1),
 ('@skelligsix18', 1),
 ('#skelligcoast2kms', 1),
 ('#southkerry', 1),
 ('https://t.co/zJcL195VQS', 1),
 ('@srinivasiyc', 1),
 ('https://t.co/IAEK4fWsgz', 1),
 ('premiership', 1),
 ('non-playing', 1),
 ('subsidise', 1),
 ('playersã', 1),
 ('renewing', 1),
 ('flew', 1),
 ('@torontopearson', 1),
 ('@680news', 1),
 ('https://t.co/7j2Y3rSld9', 1)]

Прикольно, наименее популярные это в основном странные хэштэги и слова с опечатками


**Ответ:** в данном конкретном контексте по этим четырём топам трудно оценить победителя.

В TweetTokenizer не следовало отделять 19 в отдельный токен, но при этом мы смогли лучше санитизировать "плохие" символы

Интуитивно хочется сказать, что TweetTokenizer должен быть лучше, но сейчас наш анализ этого не выявил

Выведите топ-10 популярных хештегов (токены, первые символы которых - #) с количеством встреч. Что можно сказать о них?

In [20]:
i = 0
for word, cnt in tw_token_dict.most_common():
    if word[0] == '#':
        print(word, cnt)
        i += 1
    if i >= 10:
        break

#coronavirus 8808
#covid19 2589
#covid_19 1734
#covid2019 946
#toiletpaper 744
#covid 641
#socialdistancing 465
#coronacrisis 448
#pandemic 257
#coronaviruspandemic 249


**Ответ:** разумеется, они все так или иначе связаны с ковидом, за исключением выбивающегося #toiletpaper, который показывает, что же именно волновало людей тогда

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [21]:
i = 0
for word, cnt in tw_token_dict.most_common():
    if word.startswith('https://t.co'):
        print(word, cnt)
        i += 1
    if i >= 10:
        break

https://t.co/oXA7SWtoNd 5
https://t.co/gP3EusapL8 4
https://t.co/DefTruI1PfÃÂ 3
https://t.co/WrLHYzIzAA 3
https://t.co/kuwIpF1KQW 3
https://t.co/zjNRx6dKKN 3
https://t.co/3GBBDpdjat 3
https://t.co/e2ZNXajPre 3
https://t.co/CATKegAyOY 3
https://t.co/G63RP042HO 3


**Ответ:** числа намноооого ниже (топ 1  это 8808 vs 5), из-за чего ссылки выглядят бесполезными

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для  выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа с позицией в таблице Unicode 128 и более,  ссылки на t.co



In [22]:
from string import punctuation
from nltk.corpus import stopwords

def rm_noise(tokens):
    stop_words = set(stopwords.words('english') + list(punctuation))

    for word in tokens:
        if (len(word) == 1 and ord(word) >= 128)\
                or word.startswith('https://t.co'):
            stop_words.add(word)

    new_tokens = [x for x in tokens if x not in stop_words]
    return new_tokens

In [23]:
def custom_tokenizer(text):
    tokens = TweetTokenizer(preserve_case=False).tokenize(text)
    tokens = rm_noise(tokens)

    return tokens

In [24]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=custom_tokenizer)
cv.fit(train['OriginalTweet'])

display(f'Размер изначального словаря: {len(token_dict)}')
display(f'Размер нового словаря: {len(cv.vocabulary_)}')

/home/bosbobos/.conda/envs/iad/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


'Размер изначального словаря: 79566'

'Размер нового словаря: 45290'

**Ответ:** почти вдвое сократили количество токенов, и обосновали, что качество этого не только не упадёт, а вполне себе может повыситься, ну супер же

Посмотрим на какой-нибудь конкретный твитт:

In [26]:
ind = 9023
tw, sent = train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']
tw, sent

('Nice one @SkyNews lets not panic but show ppl in france queueing for food!!! #CoronavirusOutbreak #COVID2019 brainless!! Ffs',
 np.int64(0))

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [27]:
def print_token_counts(vectorizer, sentence):
    vector = vectorizer.transform([sentence])
    nonzero_indices = vector.nonzero()[1]
    counts = vector.toarray()[0][nonzero_indices]

    index_to_word = {idx: word for word, idx in vectorizer.vocabulary_.items()}

    token_counts = {index_to_word[idx]: count for idx, count in zip(nonzero_indices, counts)}
    for w in sorted(token_counts, key=token_counts.get, reverse=True):
        print(w, token_counts[w])

In [28]:
print_token_counts(cv, tw)

#coronavirusoutbreak 1
#covid2019 1
@skynews 1
brainless 1
ffs 1
food 1
france 1
lets 1
nice 1
one 1
panic 1
ppl 1
queueing 1
show 1


**Ответ:** класс, каждое слово встречается в нашем тексте ровно единожды. Мы провели абсолютно бесполезную операцию, как же мы хороши

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)
tfidf.fit(train['OriginalTweet'])
print_token_counts(tfidf, tw)

brainless 0.3867641170466375
queueing 0.35434556333216544
ffs 0.3244741764513268
france 0.32047314046539654
@skynews 0.31042634466284263
lets 0.30121070136861045
ppl 0.2550064610386691
nice 0.2542729176022132
show 0.24406492401820118
#coronavirusoutbreak 0.22584265007428544
#covid2019 0.16825192939361902
one 0.16306722855395073
panic 0.14759470306326164
food 0.11328893069250721


**Ответ:** на этот раз нам действительно показывается что-то важное. Как мы уже выяснили, в тексте каждое слово встречается единожды, так что сейчас нам показывается лишь IDF, то есть логарифм обратной частоты встречаемости каждого слово, иными словами, коэффициент редкости.

И действительно, три слова с макс. коэф-ом явно указывают на негативную окраску текста

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [30]:
train[train['OriginalTweet'].apply(lambda x: 'incredible' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
3505,8053,53005,Canada,18-03-2020,A lot of well deserved appreciation toward our...,1
30265,40981,85933,"New York, NY",10-04-2020,Truly incredible consumer behavior graphs in t...,1
944,4938,49890,"Capital Region, NY, USA",17-03-2020,We encourage you to find safe and creative way...,1
23999,33136,78088,Atlanta area,05-04-2020,who through incredible selfishness is keeping ...,1
20783,29152,74104,"Regina, Saskatchewan",31-03-2020,Rawlco Radio has been an incredible partner of...,1
16034,23306,68258,"Finland, LatAm",24-03-2020,"People need easy, safe and immediate access se...",1
21120,29558,74510,Unknown,01-04-2020,Sending a virtual thanks round of applause to ...,1
17228,24764,69716,Brazil,25-03-2020,Hello has hit some hard times due to COVID 19 ...,1
19556,27611,72563,Chicago | Seat 21A on a plane,26-03-2020,This is an incredible tale of a forward lookin...,1
29859,40462,85414,Lowell USA,09-04-2020,They reduced the number of people allowed in t...,1


In [31]:
tw_id = 3505
msg = train.loc[tw_id]['OriginalTweet']
display(msg)
print_token_counts(tfidf, msg)

'A lot of well deserved appreciation toward our incredible health care workers. ?? I also want to give a big thanks to the grocery store clerks. Many of whom make minimum wage, and are keeping our food supply going strong. #coronavirus'

deserved 0.3092883211610608
incredible 0.27654864813481295
appreciation 0.2672772890176374
toward 0.2639515668643063
minimum 0.23924038973052159
wage 0.2377119553822904
clerks 0.23176798006143828
strong 0.21919638723034857
keeping 0.20038882011910375
lot 0.19086748898392972
give 0.1890907000498282
big 0.18799447360265067
thanks 0.18775547491480504
care 0.17663263009258473
well 0.17317335099939846
want 0.17104410052956648
health 0.16502820601296384
also 0.15947237677330256
supply 0.15837778797568033
make 0.1533673735230785
many 0.15223038637840264
going 0.14200972163059003
workers 0.13518018791906142
grocery 0.10254855559868026
store 0.09950463974843003
food 0.09888353857325324
#coronavirus 0.0691905206897339


**Ответ:** на удивление, пришлось перебрать несколько примеров, прежде чем нашелся такой удачный, но в данном примере в топ 3 есть слова incredible и appreciation, действительно имеющие явно позитивный окрас

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [32]:
count_vr = CountVectorizer(tokenizer=custom_tokenizer)
count_train = count_vr.fit_transform(train['OriginalTweet'])
count_train_y = train['Sentiment']
count_test = count_vr.transform(test['OriginalTweet'])
count_test_y = test['Sentiment']

/home/bosbobos/.conda/envs/iad/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
tfidf_vr = TfidfVectorizer(tokenizer=custom_tokenizer)
tfidf_train = tfidf_vr.fit_transform(train['OriginalTweet'])
tfidf_train_y = train['Sentiment']
tfidf_test = tfidf_vr.transform(test['OriginalTweet'])
tfidf_test_y = test['Sentiment']

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

Используйте `sparse` матрицы (после векторизации), не превращайте их в `numpy.ndarray` или `pd.DataFrame` - может не хватить памяти.

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

count_log_reg = LogisticRegression(random_state=17).fit(count_train, count_train_y)
tfidf_log_reg = LogisticRegression(random_state=17).fit(tfidf_train, tfidf_train_y)

acc_count_train = accuracy_score(count_train_y, count_log_reg.predict(count_train))
acc_count_test  = accuracy_score(count_test_y,  count_log_reg.predict(count_test))
acc_tfidf_train = accuracy_score(tfidf_train_y, tfidf_log_reg.predict(tfidf_train))
acc_tfidf_test  = accuracy_score(tfidf_test_y,  tfidf_log_reg.predict(tfidf_test))

df_results = pd.DataFrame({
    'train': [acc_count_train, acc_tfidf_train],
    'test':  [acc_count_test,  acc_tfidf_test ]
}, index=['Count', 'TF-IDF']).reset_index().rename(columns={'index':'Vectorizer'})
df_results.set_index('Vectorizer', inplace=True)
df_results

,train,test
Vectorizer,,
Count,0.984665,0.867052
TF-IDF,0.922426,0.852800


**Ответ:** на удивление, tf-idf оказался значительно хуже на обучающей (всё равно, а может и лучше, если произошло переобучение) и чуточку хуже на тестовой выборке, что достаточно странно. Отсутствие стемминга может обосновать такой результат, так что пока не сокрушаемся и едем дальше

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [35]:
from nltk.stem import SnowballStemmer

def custom_stem_tokenizer(text):
    stemmer = SnowballStemmer('english')
    tokens = TweetTokenizer(preserve_case=False).tokenize(text)
    tokens = rm_noise(tokens)
    tokens = [stemmer.stem(x) for x in tokens]

    return tokens

In [36]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [37]:
cv = CountVectorizer(tokenizer=custom_stem_tokenizer).fit(train['OriginalTweet'])

print(len(cv.vocabulary_))

/home/bosbobos/.conda/envs/iad/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


36634


**Ответ** было 45290, стало 36634, класс

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [38]:
count_vr = CountVectorizer(tokenizer=custom_stem_tokenizer)
count_train = count_vr.fit_transform(train['OriginalTweet'])
count_train_y = train['Sentiment']
count_test = count_vr.transform(test['OriginalTweet'])
count_test_y = test['Sentiment']

In [39]:
tfidf_vr = TfidfVectorizer(tokenizer=custom_stem_tokenizer)
tfidf_train = tfidf_vr.fit_transform(train['OriginalTweet'])
tfidf_train_y = train['Sentiment']
tfidf_test = tfidf_vr.transform(test['OriginalTweet'])
tfidf_test_y = test['Sentiment']

In [152]:
count_log_reg = LogisticRegression(random_state=17).fit(count_train, count_train_y)
tfidf_log_reg = LogisticRegression(random_state=17).fit(tfidf_train, tfidf_train_y)

acc_count_train = accuracy_score(count_train_y, count_log_reg.predict(count_train))
acc_count_test  = accuracy_score(count_test_y,  count_log_reg.predict(count_test))
acc_tfidf_train = accuracy_score(tfidf_train_y, tfidf_log_reg.predict(tfidf_train))
acc_tfidf_test  = accuracy_score(tfidf_test_y,  tfidf_log_reg.predict(tfidf_test))

df_results.loc['Count with Stem'] = [acc_count_train, acc_count_test]
df_results.loc['TF-IDF with Stem'] = [acc_tfidf_train, acc_tfidf_test]
df_results

,train,test
Vectorizer,,
Count,0.984665,0.867052
TF-IDF,0.922426,0.852800
Count with Stem,0.972021,0.867451
TF-IDF with Stem,0.916574,0.855990


**Ответ:** качество на тестовой выборке поднять получилось, однако Count всё ещё побеждает

Видимо, в данном случае присутствие ключевых слов, вроде amazing или awful является наиболее важным показателем, поэтому большее внимание, уделяемое им CountVectorizer, помогает модели. В то же время, считая частоту, TF-IDF наоборот принижает их значимость

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36633 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [48]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=8708
                        ).fit(train['OriginalTweet'])
print(len(cv_df.vocabulary_))

/home/bosbobos/.conda/envs/iad/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


36633


Запустим сначала с огромным max_df и увидим частоту наиболее частого токена - #coronavirus - 8809. Поставим max_df 8808 и увидим, что почему-то число не изменилось, так что для уверенности поставим 8708 и всё работает!! 

In [46]:
print_token_counts(cv_df, ' '.join(train['OriginalTweet']))

#coronavirus 8809
19 7167
covid 6253
price 5283
store 4671
food 4463
supermarket 4193
groceri 3760
peopl 3501
shop 2796
consum 2723
#covid19 2592
go 2489
get 2302
need 2189
time 1969
buy 1954
worker 1892
onlin 1833
hand 1820
like 1808
work 1803
help 1776
#covid_19 1734
sanit 1682
... 1680
panic 1672
demand 1610
pandem 1580
stock 1506
us 1436
home 1385
make 1347
suppli 1281
one 1270
coronavirus 1170
take 1166
pleas 1163
use 1132
day 1127
keep 1115
week 1102
stay 1078
crisi 1077
due 1065
see 1044
mask 1040
thank 991
busi 981
oil 962
say 951
#covid2019 946
good 941
market 936
retail 931
increas 928
new 923
product 911
shelv 888
stop 885
deliveri 883
local 871
amp 864
mani 849
toilet 847
today 841
paper 834
2 807
still 788
know 773
essenti 769
look 757
protect 756
spread 749
#toiletpap 744
would 737
close 729
even 723
servic 719
think 713
custom 711
via 708
staff 698
thing 685
everyon 684
safe 684
also 683
social 669
way 664
iã 663
come 662
outbreak 660
employe 659
want 659
order 657
virus

**Ответ:** параметр получился достаточно большой, потому что токен #coronavirus встречается в большом количестве текстов

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [55]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=11
                        ).fit(
                            train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

/home/bosbobos/.conda/envs/iad/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


3687


In [56]:
cv_df_train = cv_df.fit_transform(train['OriginalTweet'])
cv_df_train_y = train['Sentiment']
cv_df_test = cv_df.transform(test['OriginalTweet'])
cv_df_test_y = test['Sentiment']

cv_df_log_reg = LogisticRegression(random_state=17).fit(cv_df_train, cv_df_train_y)

acc_cv_df_train = accuracy_score(cv_df_train_y, cv_df_log_reg.predict(cv_df_train))
acc_cv_df_test  = accuracy_score(cv_df_test_y,  cv_df_log_reg.predict(cv_df_test))

df_results.loc['Count with min_df'] = [acc_cv_df_train, acc_cv_df_test]
df_results

/home/bosbobos/.conda/envs/iad/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,train,test
Vectorizer,,
Count,0.984665,0.867052
TF-IDF,0.922426,0.852800
Count with min_df,0.929005,0.868049


**Ответ:** ура, мы повысили качество на одну тысячную, как же мы хороши, точно всё делали не зря

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `barplot`, содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на основе целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --